In [ ]:
# Iniciando Sessão Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('Votacao_Candidato_MunZona_2022') \
    .master("spark://spark:7077") \
    .getOrCreate()

In [ ]:
# Lendo o CSV e Salvando em um DataFrame
df = spark.read.csv("jobs/votacao_candidato_munzona_2022_BRASIL.csv", sep=";", header=True, inferSchema=True, encoding="ISO-8859-1")
df.show(20)

In [ ]:
# Selecionando as colunas necessárias para o OBT
columns = ["NR_TURNO","DS_ELEICAO","TP_ABRANGENCIA","SG_UF","NM_MUNICIPIO","NR_ZONA","DS_CARGO","NR_CANDIDATO","NM_CANDIDATO","NM_URNA_CANDIDATO","DS_SITUACAO_CANDIDATURA","NR_PARTIDO","SG_PARTIDO","NM_PARTIDO","NM_COLIGACAO","DS_COMPOSICAO_COLIGACAO","ST_VOTO_EM_TRANSITO","QT_VOTOS_NOMINAIS","NM_TIPO_DESTINACAO_VOTOS","QT_VOTOS_NOMINAIS_VALIDOS","DS_SIT_TOT_TURNO"]
new_df = df.select(columns)
new_df.show(20)

In [ ]:
# Recebendo as variáveis do Banco
hostname_or_ip = "postgres"
port = "5432"
db = "obt"
user = "metabase"
password = "FlaDWaiNEdySeRiG"
schema = "obt"
table = "obt"

In [ ]:
# Ajustando as variáveis do formulário
url = "jdbc:postgresql://" + hostname_or_ip + ":" + port + "/" + db
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

In [ ]:
# Gravar dados no banco postgresql
new_df.write.jdbc(url=url, table=table, mode="overwrite", properties=properties)